In [1]:
from pip_flow.pip_flow import PipFlow


pip_flow=PipFlow(device='cloud', model_key="PipableAI/pipable-worflow-automaton-private-v2")

In [2]:
prompt="""
<question>
write a python function to double the input number.
</question>
"""
print(pip_flow.generate(prompt, eos_token="code"))



```python
def double_number(num):
  return num * 2
```




In [3]:
import requests
from bs4 import BeautifulSoup

functions = [
requests.get,
BeautifulSoup,
BeautifulSoup.find_all,
]
# docs= pip_flow.generate_docs(requests.post)
# print(docs)
pip_flow.register_callables(functions,generate_docs=False) #keeps on appending to the object

In [4]:
pip_flow.functions

[Function(signature='(url, params=None, **kwargs)', docs='Sends a GET request.\n\n    :param url: URL for the new :class:`Request` object.\n    :param params: (optional) Dictionary, list of tuples or bytes to send\n        in the query string for the :class:`Request`.\n    :param \\*\\*kwargs: Optional arguments that ``request`` takes.\n    :return: :class:`Response <Response>` object\n    :rtype: requests.Response\n    ', name='get', full_name='requests.api.get'),
 Function(signature="(markup='', features=None, builder=None, parse_only=None, from_encoding=None, exclude_encodings=None, element_classes=None, **kwargs)", docs='A data structure representing a parsed HTML or XML document.\n\n    Most of the methods you\'ll call on a BeautifulSoup object are inherited from\n    PageElement or Tag.\n\n    Internally, this class defines the basic interface called by the\n    tree builders when converting an HTML/XML document into a data\n    structure. The interface abstracts away the differe

In [5]:

config = {
"instructions": "be smarter"
}
print(pip_flow.make_live_prompt("default", config))

Updated config and base prompt template successfully fetched.


In [6]:
pip_flow.latest_config

{'func_info': '[\'--name:get\\n--annotations:(url, params=None, **kwargs)\\n--doc:Sends a GET request.\\n\\n    :param url: URL for the new :class:`Request` object.\\n    :param params: (optional) Dictionary, list of tuples or bytes to send\\n        in the query string for the :class:`Request`.\\n    :param \\\\*\\\\*kwargs: Optional arguments that ``request`` takes.\\n    :return: :class:`Response <Response>` object\\n    :rtype: requests.Response\\n    \\n\\n\', \'--name:BeautifulSoup\\n--annotations:(markup=\\\'\\\', features=None, builder=None, parse_only=None, from_encoding=None, exclude_encodings=None, element_classes=None, **kwargs)\\n--doc:A data structure representing a parsed HTML or XML document.\\n\\n    Most of the methods you\\\'ll call on a BeautifulSoup object are inherited from\\n    PageElement or Tag.\\n\\n    Internally, this class defines the basic interface called by the\\n    tree builders when converting an HTML/XML document into a data\\n    structure. The int

In [7]:
ques = "get the content of https://google.com, create beautiful soup class and find all divs in it"
plan =pip_flow.generate_plan(ques)
print(plan)

{
    "tasks": [
        {
            "task_id": 1,
            "function_name": "get",
            "parameters": [
                {
                    "name": "url",
                    "value": "https://google.com",
                    "description": "URL to be fetched",
                    "dtype": "string"
                }
            ],
            "outputs": [
                "variable_1"
            ],
            "description": "Fetches the content of the specified URL using the GET method."
        },
        {
            "task_id": 2,
            "function_name": "BeautifulSoup",
            "parameters": [
                {
                    "name": "markup",
                    "value": "variable_1",
                    "description": "Markup to be parsed",
                    "dtype": "string"
                }
            ],
            "outputs": [
                "variable_2"
            ],
            "description": "Creates a BeautifulSoup object from the parse

In [8]:
pip_flow.plan_to_code()


```python
import requests
from bs4 import BeautifulSoup

# Fetch the content of the URL using the GET method
url = 'https://google.com'
response = requests.api.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all div elements
divs = soup.find_all('div')

# Assign the BeautifulSoup object to a variable
bs = soup

# Assign the divs to a variable
divs = divs
```
